<a href="https://colab.research.google.com/github/LordLean/Acquired-Intelligence-Adaptive-Behaviour/blob/master/AIAB_Labs/microbialGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib as plt
import random

In [0]:
# (benefit, cost)
knapsack = [(5,3),(6,2),(1,4),(9,5),(2,8),(8,9),(4,10),(3,1),(7,6),(10,7)]
config = {"kp":knapsack, "capacity": 20}

In [0]:
class MicrobialGA(object):


  def __init__(self, kp, capacity, crossover_op, population_size=100, generations=100, mutation_rate=0.3):
    self.kp = kp # Knapsack to be evaluated.
    self.capacity = capacity # Knapsack capacity.
    self.population_size = population_size # Population size of genotypes.
    self.generations = generations # Evolutions to mutate for. 
    self.mutation_rate = mutation_rate # Mutation rate to effect rate of evolution.
    self.geno_shape = len(self.kp) # Shape of desired genotype.
    self.population = np.zeros(shape=(population_size, self.geno_shape), dtype=int) # Hold all genotypes/solutions.


  def initalize(self):
    # Initialze each solution in population to random binary values.
    for i, genotype in enumerate(self.population):
      current_sum = self.capacity + 1
      # Ensure random values do not cause a cost above capacity.
      while current_sum > self.capacity:
        for i, _ in enumerate(genotype):
          num = random.uniform(0,1)  
          genotype[i] = np.round(num)
          # Update current sum until it holds a value less than capacity.
          current_sum = np.sum([self.kp[i][1] * genotype[i] for i, _ in enumerate(self.kp)])


  def selection(self):
    # Create indicies to compare without out of bounds exception.
    upper_bound = self.geno_shape - 1
    index_one = random.uniform(0,upper_bound)  
    index_one = np.round(index_one).astype("int")
    index_two = int()
    p = random.uniform(0,1)
    # If p < 1/4 match genotype with previous neighbour at distance 2. 
    if p < 0.25:
      index_two = index_one - 2
    # If 1/4 <= p < 1/2 match genotype with previous neighbour at distance 1. 
    elif p < 0.5:
      index_two = index_one - 1
    # If 1/2 <= p < 3/4 match genotype with next neighbour at distance 1. 
    elif p < 0.75:
      # Try-catch to avoid out of bounds exception. If index does not exist set to 0.
      try:
        index_two = index_one + 1
        # Try catch was maybe not the best solution as had to force a possible error but this was the imp. that I had started.
        geno_one = self.population[index_one] 
        geno_two = self.population[index_two]
        return geno_one, geno_two
      except:
        index_two = 0
    # If 3/4 <= p <= 1 match genotype with next neighbour at distance 2. 
    else:
      # Try-catch to avoid out of bounds exception. If index does not exist set to 1.
      try:
        index_two = index_one + 2
        geno_one = self.population[index_one] 
        geno_two = self.population[index_two]
        return geno_one, geno_two
      except:
        index_two = 1
    # State the two "neighbouring" genotypes to return.
    geno_one = self.population[index_one] 
    geno_two = self.population[index_two] 
    return geno_one, geno_two
    

  def fitness(self, geno_one, geno_two):
    # Benefit-cost evaluation.
    geno_one_bene = np.sum([self.kp[i][0] * geno_one[i] for i, _ in enumerate(self.kp)])
    geno_one_cost = np.sum([self.kp[i][1] * geno_one[i] for i, _ in enumerate(self.kp)])
    #Benefit-cost evaluation.
    geno_two_bene = np.sum([self.kp[i][0] * geno_two[i] for i, _ in enumerate(self.kp)])
    geno_two_cost = np.sum([self.kp[i][1] * geno_two[i] for i, _ in enumerate(self.kp)])
    # Case: benefits are equal, cost is evaluated.
    if geno_one_bene == geno_two_bene:
      if geno_one_cost <= geno_two_cost:
        return geno_one
      else:
        return geno_two
    # If genotype one has a greater benefit then that genotype is returned.
    elif geno_one_bene > geno_two_bene:
      return geno_one
    # If genotype one has a lesser benefit then that genotype is returned.
    else:
      return geno_two


  def crossover(self,crossover_op):
    NotImplemented

In [184]:
mga = MicrobialGA(config["kp"], config["capacity"], None, population_size=10)
mga.initalize()
mga.population

array([[0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0]])

In [186]:
one, two = mga.selection()
mga.fitness(one,two)

array([0, 1, 0, 1, 0, 0, 1, 1, 0, 0])

In [25]:
mga = MicrobialGA(config["kp"], config["capacity"], 10)
mga.population
for i, item in enumerate(mga.population):
  current_sum = 20 + 1
  # Ensure random values do not cause a cost above capacity.
  while current_sum > 20:
    for i, _ in enumerate(item):
      num = random.uniform(0,1)  
      item[i] = np.round(num)
      # Update current sum until it holds a value less than capacity.
      current_sum =  np.sum([mga.kp[i][1] * item[i] for i, _ in enumerate(mga.kp)])
mga.population

array([[0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 1, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 0,

In [36]:
print(mga.population)
print()
print(mga.population[0])
print(mga.population[-1])

[[1 1 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 1 0 0]
 [0 0 1 1 0 0 0 1 0 0]
 [0 1 1 0 0 1 0 0 0 0]
 [0 1 0 1 0 0 0 1 0 1]
 [1 0 0 0 0 1 0 1 0 0]
 [0 1 0 0 1 0 0 1 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [1 1 0 0 1 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 1 0]]

[1 1 0 0 0 0 1 0 0 0]
[0 0 1 0 0 0 0 0 1 0]


In [131]:
test = np.zeros(10,dtype=int)
upper_bound = 10 - 1
num = random.uniform(0,upper_bound)  
num = np.round(num).astype("int")
num = num + 1
test[0] = 999
try:
  print(test[num])
except:
  print(num)
  print(test[0])

9.0
999
